# Diffie-Hellman Man-in-the-Middle (MITM) Attack Simulation

## Introduction
The Diffie-Hellman (DH) key exchange algorithm allows two parties, Alice and Bob, to establish a shared secret over an insecure channel. However, the standard DH protocol does not authenticate the participants. This vulnerability allows an attacker, Mallory, to intercept the communication and perform a Man-in-the-Middle (MITM) attack.

In this notebook, we will simulate this attack where Mallory intercepts the public keys exchanged between Alice and Bob and replaces them with her own public key derived from a private key $m=10$.

In [ ]:
import random

# 1. Setup: Define Public Parameters
p = 23  # A prime number
g = 5   # A generator modulo p

print(f"Public Parameters:\n  Prime (p): {p}\n  Generator (g): {g}\n")

Public Parameters:
  Prime (p): 23
  Generator (g): 5



## Step 1: Alice Generates Her Keys
Alice generates a private key $a$ and calculates her public key $A = g^a \pmod p$. She intends to send $A$ to Bob.

In [2]:
# Alice's Keys
a = random.randint(1, p-1)  # Alice's private key
A = pow(g, a, p)            # Alice's public key

print(f"Alice:\n  Private Key (a): {a}\n  Public Key (A): {A}\n")

Alice:
  Private Key (a): 16
  Public Key (A): 3



## Step 2: Mallory Intercepts Alice's Key
Mallory intercepts Alice's public key $A$. Instead of forwarding $A$ to Bob, she generates her own private key $m=10$ and calculates a public key $M = g^m \pmod p$. She sends $M$ to Bob, pretending to be Alice.

In [3]:
# Mallory's Keys
m = 10                      # Mallory's chosen private key
M = pow(g, m, p)            # Mallory's public key

print(f"Mallory (MITM):\n  Intercepted A: {A}\n  Mallory's Private Key (m): {m}\n  Mallory's Public Key (M): {M}\n")
print("Mallory sends M to Bob (pretending to be Alice)...\n")

Mallory (MITM):
  Intercepted A: 3
  Mallory's Private Key (m): 10
  Mallory's Public Key (M): 9

Mallory sends M to Bob (pretending to be Alice)...



## Step 3: Bob Generates His Keys
Bob receives $M$ (thinking it is from Alice). He generates his own private key $b$ and calculates his public key $B = g^b \pmod p$. He sends $B$ back, intending it for Alice.

In [4]:
# Bob's Keys
b = random.randint(1, p-1)  # Bob's private key
B = pow(g, b, p)            # Bob's public key

print(f"Bob:\n  Received Key (M): {M}\n  Private Key (b): {b}\n  Public Key (B): {B}\n")

Bob:
  Received Key (M): 9
  Private Key (b): 8
  Public Key (B): 16



## Step 4: Mallory Intercepts Bob's Key
Mallory intercepts Bob's public key $B$. She prevents it from reaching Alice and instead sends her own public key $M$ to Alice, pretending to be Bob.

In [5]:
print(f"Mallory (MITM):\n  Intercepted B: {B}\n")
print("Mallory sends M to Alice (pretending to be Bob)...\n")

Mallory (MITM):
  Intercepted B: 16

Mallory sends M to Alice (pretending to be Bob)...



## Step 5: Calculating Shared Secrets
Now, everyone calculates their shared secrets.

1. **Alice** computes $S_A$ using the key she received ($M$) and her private key ($a$): $S_A = M^a \pmod p$.
2. **Bob** computes $S_B$ using the key he received ($M$) and his private key ($b$): $S_B = M^b \pmod p$.
3. **Mallory** computes two secrets:
    - $S_{MA}$ (shared with Alice) using Alice's key ($A$) and her private key ($m$): $S_{MA} = A^m \pmod p$.
    - $S_{MB}$ (shared with Bob) using Bob's key ($B$) and her private key ($m$): $S_{MB} = B^m \pmod p$.

In [6]:
# Alice calculates secret with Mallory (thinking it's Bob)
S_A = pow(M, a, p)

# Bob calculates secret with Mallory (thinking it's Alice)
S_B = pow(M, b, p)

# Mallory calculates secret with Alice
S_MA = pow(A, m, p)

# Mallory calculates secret with Bob
S_MB = pow(B, m, p)

print("Calculated Shared Secrets:")
print(f"  Alice's Secret (S_A): {S_A}")
print(f"  Bob's Secret   (S_B): {S_B}")
print(f"  Mallory-Alice  (S_MA): {S_MA}")
print(f"  Mallory-Bob    (S_MB): {S_MB}")

Calculated Shared Secrets:
  Alice's Secret (S_A): 8
  Bob's Secret   (S_B): 13
  Mallory-Alice  (S_MA): 8
  Mallory-Bob    (S_MB): 13


## Step 6: Verification and Analysis

Let's verify the results of the attack:
1. **Alice and Bob do NOT share the same secret**: $S_A \neq S_B$.
2. **Mallory shares a secret with Alice**: $S_A = S_{MA}$.
3. **Mallory shares a secret with Bob**: $S_B = S_{MB}$.

In [7]:
print("\nVerification:")
print(f"  Alice and Bob share same secret? {S_A == S_B}")
print(f"  Mallory can decrypt Alice's messages? {S_A == S_MA}")
print(f"  Mallory can decrypt Bob's messages?   {S_B == S_MB}")

if S_A == S_MA and S_B == S_MB:
    print("\n[SUCCESS] MITM Attack Successful! Mallory controls the channel.")
else:
    print("\n[FAILURE] Something went wrong with the simulation.")


Verification:
  Alice and Bob share same secret? False
  Mallory can decrypt Alice's messages? True
  Mallory can decrypt Bob's messages?   True

[SUCCESS] MITM Attack Successful! Mallory controls the channel.


## Analysis: Why did Diffie-Hellman fail?

The standard Diffie-Hellman key exchange protocol failed to prevent this attack because it lacks **Authentication**.

- **No Identity Verification**: Alice has no way to verify that the public key she received actually belongs to Bob. Similarly, Bob cannot verify the source of the key he received.
- **Trust**: The protocol assumes an authenticated channel or that the public keys are exchanged securely, which is not the case here.

### Solution
To prevent MITM attacks, Diffie-Hellman is usually combined with an authentication mechanism, such as:
1. **Digital Signatures**: Alice and Bob sign their public keys with their private signing keys. They verify each other's signatures using trusted public keys (PKI).
2. **Certificates**: Using a Certificate Authority (CA) to bind public keys to identities.